In [5]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_variable_cost_runs(acq, dim):
    runs = api.runs(path="ziv-scully-group/Gittins for Bayesian Optimization", filters={
        "sweep": "h3xxnbzz", 
        "config.lengthscale": 0.1, 
        "config.amplitude": 1, 
        "config.kernel": "matern52", 
        "config.dim": dim,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global_optimum_value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [6]:
acquisition_functions = {
      'ExpectedImprovement':'ExpectedImprovementPerCost',
      'ExpectedImprovementWithCost_Uniform':'ExpectedImprovementPerCost_Uniform',
      'ExpectedImprovementWithCost_Cooling':'ExpectedImprovementPerCost_Cooling',
      'Gittins_Lambda_01':'Gittins_Lambda01',
      'Gittins_Lambda_001':'Gittins_Lambda001',
      'Gittins_Lambda_0001':'Gittins_Lambda0001',
      'Gittins_Step_Divide2':'Gittins_Step_Divide2',
      'Gittins_Step_Divide5':'Gittins_Step_Divide5',
      'Gittins_Step_Divide10':'Gittins_Step_Divide10',
      'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
dimensions = [4,8,16]

In [7]:
grouped_runs = {(a,d): load_bayesian_regret_variable_cost_runs(a,d) for a in acquisition_functions.keys() for d in (dimensions)}

100%|███████████████████████████████████████████| 16/16 [00:13<00:00,  1.20it/s]


In [4]:
import numpy as np

for a in acquisition_functions.keys():
    for d in dimensions:
        config_and_metrics_per_seed = grouped_runs[a, d]

        # Initialize lists to collect filtered data
        filtered_cumulative_cost = []
        filtered_best_observed = []
        filtered_global_optimum = []

        for _, m in config_and_metrics_per_seed:
            # Assuming each 'cumulative cost' and 'best observed' are lists or arrays
            cumulative_cost = m['cumulative cost']
            best_observed = m['best observed']

            # Check if both lists are long enough to support indexing at the last desired index
            if len(cumulative_cost) > 0 and len(best_observed) > 0:
                # Select elements at indices 0, 10, 20, ..., up to the length of the array
                indices = [i for i in range(0, len(cumulative_cost), 10)]
                filtered_cumulative_cost.append([cumulative_cost[i] for i in indices])
                filtered_best_observed.append([best_observed[i] for i in indices])
                filtered_global_optimum.append(m['global_optimum_value'][0])

        if filtered_cumulative_cost and filtered_best_observed:  # Proceed only if there's filtered data
            cumulative_cost_per_seed = np.array(filtered_cumulative_cost).T
            best_observed_per_seed = np.array(filtered_best_observed).T
            global_optimum_per_seed = np.array(filtered_global_optimum)

            regret_per_seed = global_optimum_per_seed - best_observed_per_seed
            print(a, d, regret_per_seed.shape)

            regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
            regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
            regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

            output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75), axis=-1)

            np.savetxt(f"results/bayesian_regret/BayesianRegret_LinearCost_d{d}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

ExpectedImprovement 4 (81, 16)
ExpectedImprovement 8 (161, 16)
ExpectedImprovement 16 (321, 16)
ExpectedImprovementWithCost_Uniform 4 (81, 16)
ExpectedImprovementWithCost_Uniform 8 (161, 16)
ExpectedImprovementWithCost_Uniform 16 (321, 16)
ExpectedImprovementWithCost_Cooling 4 (81, 16)
ExpectedImprovementWithCost_Cooling 8 (161, 16)
ExpectedImprovementWithCost_Cooling 16 (321, 15)
Gittins_Lambda_01 4 (81, 16)
Gittins_Lambda_01 8 (161, 16)
Gittins_Lambda_01 16 (321, 5)
Gittins_Lambda_001 4 (81, 16)
Gittins_Lambda_001 8 (161, 16)
Gittins_Lambda_001 16 (321, 10)
Gittins_Lambda_0001 4 (81, 16)
Gittins_Lambda_0001 8 (161, 16)
Gittins_Lambda_0001 16 (321, 10)
Gittins_Step_Divide2 4 (81, 16)
Gittins_Step_Divide2 8 (161, 16)
Gittins_Step_Divide2 16 (321, 11)
Gittins_Step_Divide5 4 (81, 16)
Gittins_Step_Divide5 8 (161, 16)
Gittins_Step_Divide5 16 (321, 4)
Gittins_Step_Divide10 4 (81, 16)
Gittins_Step_Divide10 8 (161, 16)
Gittins_Step_Divide10 16 (321, 3)
Gittins_Step_EIpu 4 (81, 16)
Gittins_Ste